## **Sentiment Analysis**

In [1]:
import pandas as pd
import numpy as np

pd.set_option('max_colwidth',None)

In [2]:
df = pd.read_csv('../Data/movie_reviews_sentiment.csv')

In [3]:
df.head(2)

,movie_title,rating,genre,in_theaters_date,movie_info,directors,director_gender,tomatometer_rating,audience_rating,critics_consensus,sentiment_vader
0,A Dog's Journey,PG,"Drama, Kids & Family",5/17/19,"Bailey (voiced again by Josh Gad) is living the good life on the Michigan farm of his ""boy,"" Ethan (Dennis Quaid) and Ethan's wife Hannah (Marg Helgenberger). He even has a new playmate: Ethan and Hannah's baby granddaughter, CJ. The problem is that CJ's mom, Gloria (Betty Gilpin), decides to take CJ away. As Bailey's soul prepares to leave this life for a new one, he makes a promise to Ethan to find CJ and protect her at any cost. Thus begins Bailey's adventure through multiple lives filled with love, friendship and devotion as he, CJ (Kathryn Prescott), and CJ's best friend Trent (Henry Lau) experience joy and heartbreak, music and laughter, and few really good belly rubs.",Gail Mancuso,female,50,92,"A Dog's Journey is as sentimental as one might expect, but even cynical viewers may find their ability to resist shedding a tear stretched to the puppermost limit.",0.9837
1,A Dog's Way Home,PG,Drama,1/11/19,"Separated from her owner, a dog sets off on an 400-mile journey to get back to the safety and security of the place she calls home. Along the way, she meets a series of new friends and manages to bring a little bit of comfort and joy to their lives.",Charles Martin Smith,male,60,71,"A Dog's Way Home may not quite be a family-friendly animal drama fan's best friend, but this canine adventure is no less heartwarming for its familiarity.",0.9237


In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
from transformers import pipeline, logging

logging.set_verbosity_error()

### **Sentiment Analysis**

In [6]:
sentiment_analyzer = pipeline(
    task='sentiment-analysis',
    model='distilbert/distilbert-base-uncased-finetuned-sst-2-english',
    device='mps' #Use GPU instead of CPU (Available only for macos arm64)
)

In [7]:
%%time

sentiment_score = df['movie_info'].apply(sentiment_analyzer)

CPU times: user 6.34 s, sys: 650 ms, total: 6.99 s
Wall time: 7.41 s


In [8]:
## Alternate way
results = sentiment_analyzer(df['movie_info'].to_list())

In [9]:
df['label_hf'] = [r['label'] for r in results]
df['sentiment_score_hf'] = [r['score'] for r in results]

In [10]:
df.head(2)

,movie_title,rating,genre,in_theaters_date,movie_info,directors,director_gender,tomatometer_rating,audience_rating,critics_consensus,sentiment_vader,label_hf,sentiment_score_hf
0,A Dog's Journey,PG,"Drama, Kids & Family",5/17/19,"Bailey (voiced again by Josh Gad) is living the good life on the Michigan farm of his ""boy,"" Ethan (Dennis Quaid) and Ethan's wife Hannah (Marg Helgenberger). He even has a new playmate: Ethan and Hannah's baby granddaughter, CJ. The problem is that CJ's mom, Gloria (Betty Gilpin), decides to take CJ away. As Bailey's soul prepares to leave this life for a new one, he makes a promise to Ethan to find CJ and protect her at any cost. Thus begins Bailey's adventure through multiple lives filled with love, friendship and devotion as he, CJ (Kathryn Prescott), and CJ's best friend Trent (Henry Lau) experience joy and heartbreak, music and laughter, and few really good belly rubs.",Gail Mancuso,female,50,92,"A Dog's Journey is as sentimental as one might expect, but even cynical viewers may find their ability to resist shedding a tear stretched to the puppermost limit.",0.9837,POSITIVE,0.998247
1,A Dog's Way Home,PG,Drama,1/11/19,"Separated from her owner, a dog sets off on an 400-mile journey to get back to the safety and security of the place she calls home. Along the way, she meets a series of new friends and manages to bring a little bit of comfort and joy to their lives.",Charles Martin Smith,male,60,71,"A Dog's Way Home may not quite be a family-friendly animal drama fan's best friend, but this canine adventure is no less heartwarming for its familiarity.",0.9237,POSITIVE,0.999534


In [11]:
## Convert the scores to positive or negative based on the label
df['sentiment_hf'] = df.apply(
    lambda row: row['sentiment_score_hf'] if row['label_hf'] == 'POSITIVE' 
            else -row['sentiment_score_hf'],
    axis=1
)

In [12]:
df_comp = df[['movie_title','movie_info','sentiment_vader','label_hf','sentiment_hf']]

In [13]:
df_comp[df_comp['sentiment_vader'] * df_comp['sentiment_hf'] < 0]

,movie_title,movie_info,sentiment_vader,label_hf,sentiment_hf
3,A Vigilante,"A once abused woman, Sadie (Olivia Wilde), devotes herself to ridding victims of their domestic abusers while hunting down the husband she must kill to truly be free. A Vigilante is a thriller inspired by the strength and bravery of real domestic abuse survivors and the incredible obstacles to safety they face.",-0.0334,POSITIVE,0.999460
7,All Is True,"The year is 1613, Shakespeare is acknowledged as the greatest writer of the age. But disaster strikes when his renowned Globe Theatre burns to the ground, and devastated, Shakespeare returns to Stratford, where he must face a troubled past and a neglected family. Haunted by the death of his only son Hamnet, he struggles to mend the broken relationships with his wife and daughters. In so doing, he is ruthlessly forced to examine his own failings as husband and father. His very personal search for the truth uncovers secrets and lies within a family at war.",-0.9955,POSITIVE,0.520498
9,American Factory,"In post-industrial Ohio, a Chinese billionaire opens a new factory in the husk of an abandoned General Motors plant, hiring two thousand blue-collar Americans. Early days of hope and optimism give way to setbacks as high-tech China clashes with working-class America.",0.5267,NEGATIVE,-0.986457
10,Among Wolves,"In the mountains where they once fought, a motorcycle club led by Bosnian war veterans defends a herd of wild horses and discovers a new kind of freedom for themselves.",-0.3167,POSITIVE,0.998941
11,Angel of Mine,"Noomi Rapace (The Girl with the Dragon Tattoo) stars as a woman on the edge in this intense psychological thriller. Having suffered a tragic loss years earlier, Lizzie (Rapace) is trying to rebuild her life when she suddenly becomes obsessed with a neighbor's daughter, believing the girl to be her own child. As Lizzie's shocking, threatening acts grow increasingly dangerous, they lead to an explosive confrontation with the girl's angry, defensive mother (Yvonne Strahovski, ""The Handmaid's Tale"").",-0.9687,POSITIVE,0.997053
...,...,...,...,...,...
157,Under the Silver Lake,"Sam (Andrew Garfield) is a disenchanted 33-year-old who discovers a mysterious woman, Sarah (Riley Keough), frolicking in his apartment's swimming pool. When she vanishes, Sam embarks on a surreal quest across Los Angeles to decode the secret behind her disappearance, leading him into the murkiest depths of mystery, scandal, and conspiracy in the City of Angels.",-0.7430,POSITIVE,0.995023
160,Us,"Haunted by an unexplainable and unresolved trauma from her past and compounded by a string of eerie coincidences, Adelaide feels her paranoia elevate to high-alert as she grows increasingly certain that something bad is going to befall her family. After spending a tense beach day with their friends, the Tylers (Emmy winner Elisabeth Moss, Tim Heidecker, Cali Sheldon, Noelle Sheldon), Adelaide and her family return to their vacation home. When darkness falls, the Wilsons discover the silhouette of four figures holding hands as they stand in the driveway. Us pits an endearing American family against a terrifying and uncanny opponent: doppelgängers of themselves.",-0.9001,POSITIVE,0.927926
161,Velvet Buzzsaw,"Velvet Buzzsaw is a satirical thriller set in the contemporary art world scene of Los Angeles, where big money artists and mega-collectors pay a high price when art collides with commerce.",-0.2732,POSITIVE,0.998773
163,Wild Rose,"WILD ROSE tells the complicated story of Rose-Lynn, a woman on a quest to become a country music star, while also grappling with the responsibilities of being recently released from prison and a young mother of two children.",-0.5106,POSITIVE,0.998910
